# Scrape 2019

Gets the NFL schedule for 2019. Interested in Week#, Game#, Ateam(home team), Bteam(visiting team), Date of game.

Data Source: http://www.nfl.com/schedules/2019/REG1

Updates scoresPlus with basic information on the schedule information for each scheduled game;
that is, year, week, game, aTeam, bTeam, and winner (='predict')


?????   Additionally, update to current year's quarterback and ranking: https://www.thedelite.com/2019-nfl-starting-quarterback-rankings/32/  **** add this 

   


### Import libraries

In [1]:
import sqlite3
from sqlite3 import Error
import re
import string
import pandas as pd

## Key inputs (variables)

In [2]:
yearX = '2019'
weekX = '12'                      # the week for new schedule

In [3]:
# Imports, etc.
import urllib.request, urllib.parse, urllib.error
from urllib.request import urlopen
import ssl
from bs4 import BeautifulSoup
import re
from datetime import datetime

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

### Connect to database

In [4]:
def create_connection(db_file):
    """ 
    Create a database connection to a SQLite database.
    Note: When you connect to an SQLite database file that does not exist, SQLite creates a new database.
    """
    try:
        conn = sqlite3.connect(db_file)
        print('Database created:', db_file, ';  Sqlite3 version:', sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()

In [5]:
# This will connect to file, if it exists.
NFLdb = "NFL.sqlite"
create_connection(NFLdb)
conn = sqlite3.connect(NFLdb)         # make  “connection” to the database
cur = conn.cursor()                            # cursor is like a ﬁle handle

Database created: NFL.sqlite ;  Sqlite3 version: 2.6.0


In [6]:
def clean_text(x):
    x = str(x)
    for punct in ['"','\n']:
        x = x.replace(punct, '')
    return x


def extractWK(yr_x, wk_x):
    """
    Extracts weekly game data from ESPN website (ususally 16 games)
    Input is year and week #
    Updates scoresPlus with: year, week, game, aTeam, and bTeam
    Returns none
    """
        
    url = 'http://www.nfl.com/schedules/2019/REG'  + wk_x 
    print(url)
    
    html = urlopen(url, context=ctx).read()
    soup = BeautifulSoup(html, "html.parser")
    print("Title:", soup.title, '****Week:', wk_x)                      # print title
    dump = soup.get_text()                        # dump data; print it to study data; also, print(soup.prettify())
    print('original dump:', len(dump))

    dump = clean_text(soup)
    temp = re.sub(r'\b(a|an|the)\b', ' ', dump)   # get rid of special articles, etc.
    temp = temp.replace('49er', 'Xxer')          # make 49ers recognizable
    temp = re.findall('\|([A-Z][a-z]+\_[A-Z][a-z]+)?\|[A-Z]{2,}', temp)
    print ('\nYear, Week,Game,bteam,ateam: ')
    for idx, item in enumerate(temp):  
        if 'Xxers' in item:
            item = item.replace('Xxers', '49ers')
        bteam = item.split('_')[0]
        ateam = item.split('_')[1]
        game = idx+1
        print(yr_x, wk_x, game, bteam, ateam)
        
        # update database with basic data for this game 
        winner = 'predict'
        
        cur.execute('INSERT or REPLACE INTO scoresPlus (year,week,game, aTeam, bTeam, winner) VALUES (?,?,?,?,?,?)',
                (yr_x, wk_x, game, ateam, bteam, winner) )

    return 

extractWK(yearX, weekX)
conn.commit()                     # Make sure it is flushed to database
conn.close()

http://www.nfl.com/schedules/2019/REG12
Title: <title>NFL 2019 Regular Season Week 12 Schedule - NFL.com</title> ****Week: 12
original dump: 235456

Year, Week,Game,bteam,ateam: 
2019 12 1 Colts Texans
2019 12 2 Broncos Bills
2019 12 3 Giants Bears
2019 12 4 Steelers Bengals
2019 12 5 Dolphins Browns
2019 12 6 Buccaneers Falcons
2019 12 7 Panthers Saints
2019 12 8 Seahawks Eagles
2019 12 9 Lions Redskins
2019 12 10 Raiders Jets
2019 12 11 Jaguars Titans
2019 12 12 Cowboys Patriots
2019 12 13 Packers 49ers
2019 12 14 Ravens Rams
